# Playing around with navier stokes in underworld 3

In [1]:
import os
import petsc4py
import underworld3 as uw
import numpy as np
import sympy
import mpi4py
import pygmsh

In [2]:
resolution = 0.1

if uw.mpi.rank == 0:

    # Generate local mesh on boss process

    with pygmsh.geo.Geometry() as geom:

        geom.characteristic_length_max = resolution

        inclusion = geom.add_circle(
            (0.5, 2.5, 0.0), 0.3, make_surface=False, mesh_size=resolution
        )
        domain = geom.add_rectangle(
            xmin=0.0,
            ymin=0.0,
            xmax=5,
            ymax=5,
            z=0,
            holes=[inclusion],
            mesh_size=resolution,
        )

        geom.add_physical(domain.surface.curve_loop.curves[0], label="bottom")
        geom.add_physical(domain.surface.curve_loop.curves[1], label="right")
        geom.add_physical(domain.surface.curve_loop.curves[2], label="top")
        geom.add_physical(domain.surface.curve_loop.curves[3], label="left")
        geom.add_physical(inclusion.curve_loop.curves, label="inclusion")
        geom.add_physical(domain.surface, label="Elements")

        geom.generate_mesh(dim=2, verbose=False)
        geom.save_geometry(f".meshes/ns_pipe_flow_{resolution}.msh")

mesh = uw.discretisation.Mesh(f".meshes/ns_pipe_flow_{resolution}.msh", 
                                  markVertices=True, 
                                  useMultipleTags=True, 
                                  useRegions=True,
                                  qdegree=3)
mesh.dm.view()

DM Object: uw_mesh 1 MPI process
  type: plex
uw_mesh in 2 dimensions:
  Number of 0-cells per rank: 3178
  Number of 1-cells per rank: 9313
  Number of 2-cells per rank: 6135
Labels:
  depth: 3 strata with value/size (0 (3178), 1 (9313), 2 (6135))
  All_Boundaries: 1 strata with value/size (1001 (221))
  Elements: 1 strata with value/size (6 (9092))
  bottom: 1 strata with value/size (1 (99))
  celltype: 3 strata with value/size (0 (3178), 1 (9313), 3 (6135))
  inclusion: 1 strata with value/size (5 (39))
  left: 1 strata with value/size (4 (99))
  right: 1 strata with value/size (2 (99))
  top: 1 strata with value/size (3 (99))


In [3]:

##mesh = uw.meshing.UnstructuredSimplexBox(minCoords=(0,0), maxCoords=(5,1), cellSize=1/10, qdegree=3)

v = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
p = uw.discretisation.MeshVariable("P", mesh, 1, degree=2)
v_star = uw.discretisation.MeshVariable("Us", mesh, mesh.dim, degree=2)

ns = uw.systems.NavierStokesSwarm(
    mesh,
    velocityField = v,
    pressureField = p,
    velocityStar_fn = v_star.sym,
)

ns.add_dirichlet_bc( (0.0, 0.0), ["top", "bottom"], (0, 1) )
ns.add_dirichlet_bc( (1.0, 0.0), "left", (0, 1) )
ns.add_dirichlet_bc( (0.0, 0.0), "inclusion", (0, 1)) 

ns.bodyforce = sympy.Matrix([0.0, 0.0])
ns.constitutive_model = uw.systems.constitutive_models.ViscousFlowModel(mesh.dim)
ns.constitutive_model.Parameters.viscosity = 1

ns.saddle_preconditioner = 1.0 / ns.constitutive_model.Parameters.viscosity


In [4]:

def plot(mesh, v, ns,step):
    
    if mpi4py.MPI.COMM_WORLD.size == 1:
        import numpy as np
        import pyvista as pv
        import vtk
        
        pv.start_xvfb()
        pv.global_theme.background = "white"
        pv.global_theme.window_size = [750, 1200]
        pv.global_theme.antialiasing = True
        pv.global_theme.jupyter_backend = "panel"
        pv.global_theme.smooth_shading = True
        mesh.vtk("tmp_mesh.vtk")
        pvmesh = pv.read("tmp_mesh.vtk")
        pvmesh.point_data["P"] = uw.function.evaluate(p.sym[0], mesh.data)
        pvmesh.point_data["V"] = uw.function.evaluate(v.sym.dot(v.sym), mesh.data)
        pvmesh.point_data["V_Star"] = uw.function.evaluate(v_star.sym.dot(v_star.sym), mesh.data)
        arrow_loc = np.zeros((ns.u.coords.shape[0], 3))
        arrow_loc[:, 0:2] = ns.u.coords[...]
        arrow_length = np.zeros((ns.u.coords.shape[0], 3))
        arrow_length[:, 0] = uw.function.evaluate(ns.u.sym[0], ns.u.coords)*0.01
        arrow_length[:, 1] = uw.function.evaluate(ns.u.sym[1], ns.u.coords)*0.01
        pl = pv.Plotter(window_size=[1000, 1000])
        pl.add_axes()
        pl.add_mesh(
            pvmesh,
            cmap="coolwarm",
            edge_color="Black",
            show_edges=True,
            scalars="V_Star",
            use_transparency=False,
            opacity=1.0,
        )
        pl.add_arrows(arrow_loc, arrow_length, mag=3)
        pl.show(cpos="xy")
        
        pl.screenshot(str(step)+".png")

In [5]:
dt = 0.01
ns.solve(timestep=dt)
for index in range(10):
    plot(mesh, v, ns, index)
    dt = ns.estimate_dt()
    print(dt)
    ns.solve(timestep=dt, zero_init_guess=False)

  0 SNES Function norm 32.0885 
  1 SNES Function norm 3.49834e-07 
Nonlinear NStokes_swarm_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_5303/3846753669.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'7dd5539b-98de-4f4f-bc65-4a1001296c5c': {'version…

0.009712206861110132
  0 SNES Function norm 3.49834e-07 
  1 SNES Function norm < 1.e-11
Nonlinear NStokes_swarm_1_ solve converged due to CONVERGED_FNORM_RELATIVE iterations 1


...

/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2577: PyVistaDeprecationWarning: antialising is deprecated.  Please use `anti_aliasing` instead.
  warnings.warn(
/Users/maxwilliams/anaconda3/envs/uw3/lib/python3.10/site-packages/pyvista/themes.py:2545: PyVistaDeprecationWarning: `anti_aliasing` is now a string or None and must be either "ssaa", "msaa", "fxaa", or None
  warnings.warn(


/var/folders/s9/rd03rq4x0px79z1jnyqlz3540000gn/T/ipykernel_5303/3846753669.py:12: PyVistaDeprecationWarning: `panel` backend is deprecated and is planned for future removal.
  pv.global_theme.jupyter_backend = "panel"


BokehModel(combine_events=True, render_bundle={'docs_json': {'b0c6f1c4-cbc2-4c93-8dca-b9bc0aeaa1c2': {'version…

0.009712203592385128
  0 SNES Function norm < 1.e-11
Nonlinear NStokes_swarm_1_ solve converged due to CONVERGED_SNORM_RELATIVE iterations 0


: 